# Project Title
### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to make available an analytical database data about League of Legends matches so insights can be taken.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
from datetime import datetime
import pandas as pd
import matplotlib as mat
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import from_json, col, udf, schema_of_json, lit, explode
from pyspark.sql import functions as  F
from pyspark.sql.types import TimestampType

pd.set_option("display.max_columns", 150)
pd.set_option("display.max_colwidth", 250)

### Step 1: Scope the Project and Gather Data

#### Scope 
> Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

- This project aims to gather and make available detailed information about League of Legend's matches so analytical insights can serve as a guide for gameplans creation and strategies validation. Answering questions like the following ones:
    - What would be the best starting items given the curren champions and players' levels?
    - Given a specific players distribution accross lanes and players' levels, where would the first gank more likely to be successfull?
    - Given a specific game stats at a certain moment at the game, what would be the best team strategy? Push or retract?
    - Given the amount of lasthits of a player, what are their for kills/deaths/assists stats likely to be?
    
- The data used for this project is a report on game matches regarding their players' specific stats and team work details;
- The end solution of this project is a data pipeline that retrieves data from the API above, stages it into Amazon S3, processes data with Amazon EMR, and places it in an analytical database which I'd use Amazon Redshift;

#### Describe and Gather Data 
> Describe the data sets you're using. Where did it come from? What type of information is included?

- The data used for this project comes from [Riot's open API](https://developer.riotgames.com/apis);
- The information on `matches*.json` dataset is about game occorrences (fact table);
- The information on `A.json` dataset is about champions details (dimension table);
- [Data Dragon API](https://developer.riotgames.com/docs/lol)
- [Champions data](http://ddragon.leagueoflegends.com/cdn/10.13.1/data/en_US/champion.json)
    - [Champion details data](http://ddragon.leagueoflegends.com/cdn/10.13.1/data/en_US/champion/Aatrox.json)
        - `Aatrox` is the name of one champion;
- [Items](http://ddragon.leagueoflegends.com/cdn/10.13.1/data/en_US/item.json)

In [2]:
spark = SparkSession.builder\
.master("local[*]")\
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
.config("spark.driver.memory", "10g")\
.getOrCreate()

spark.conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

In [16]:
# Read in the data here
raw_data = spark.read.json('../riot-scraper/results/matches')
raw_data.count()

4132

As table

In [17]:
raw_data.registerTempTable("matches")

In [18]:
sql_context = SQLContext(sparkContext=spark)

In [19]:
sql_context.sql("Select * from matches LIMIT 1").toPandas()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams
0,1497568122350,2833,2525196351,CLASSIC,MATCHED_GAME,7.12.190.9002,11,"[(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1)), (2, (5_MnglWKGij6y...","[(11, BRONZE, [Row(masteryId=6111, rank=5), Row(masteryId=6121, rank=1), Row(masteryId=6134, rank=5), Row(masteryId=6142, rank=1), Row(masteryId=6312, rank=5), Row(masteryId=6323, rank=1), Row(masteryId=6331, rank=5), Row(masteryId=6343, rank=1),...",NA1,420,9,"[([Row(championId=17, pickTurn=1), Row(championId=51, pickTurn=2), Row(championId=105, pickTurn=3), Row(championId=86, pickTurn=4), Row(championId=72, pickTurn=5)], 0, 0, 3, False, True, True, False, False, True, 0, 0, 100, 4, 0, Fail), ([Row(cha..."


In [20]:
participant_identities = raw_data.limit(1).withColumn(
    "participantIdentity",
    F.explode(F.col("participantIdentities"))
)

participant_identities.toPandas()
# raw_data.limit(1).select("gameId").union(participant_identities).toPandas()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams,participantIdentity
0,1497568122350,2833,2525196351,CLASSIC,MATCHED_GAME,7.12.190.9002,11,"[(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1)), (2, (5_MnglWKGij6y...","[(11, BRONZE, [Row(masteryId=6111, rank=5), Row(masteryId=6121, rank=1), Row(masteryId=6134, rank=5), Row(masteryId=6142, rank=1), Row(masteryId=6312, rank=5), Row(masteryId=6323, rank=1), Row(masteryId=6331, rank=5), Row(masteryId=6343, rank=1),...",NA1,420,9,"[([Row(championId=17, pickTurn=1), Row(championId=51, pickTurn=2), Row(championId=105, pickTurn=3), Row(championId=86, pickTurn=4), Row(championId=72, pickTurn=5)], 0, 0, 3, False, True, True, False, False, True, 0, 0, 100, 4, 0, Fail), ([Row(cha...","(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1))"
1,1497568122350,2833,2525196351,CLASSIC,MATCHED_GAME,7.12.190.9002,11,"[(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1)), (2, (5_MnglWKGij6y...","[(11, BRONZE, [Row(masteryId=6111, rank=5), Row(masteryId=6121, rank=1), Row(masteryId=6134, rank=5), Row(masteryId=6142, rank=1), Row(masteryId=6312, rank=5), Row(masteryId=6323, rank=1), Row(masteryId=6331, rank=5), Row(masteryId=6343, rank=1),...",NA1,420,9,"[([Row(championId=17, pickTurn=1), Row(championId=51, pickTurn=2), Row(championId=105, pickTurn=3), Row(championId=86, pickTurn=4), Row(championId=72, pickTurn=5)], 0, 0, 3, False, True, True, False, False, True, 0, 0, 100, 4, 0, Fail), ([Row(cha...","(2, (5_MnglWKGij6y8l7Y442WzlGxoZSj8DcY569YUKVMmLBsA, 5_MnglWKGij6y8l7Y442WzlGxoZSj8DcY569YUKVMmLBsA, NA1, /v1/stats/player_history/NA1/51360237, NA1, 23, pX0DANDsG8wcP4lXWURjDU4pfDI53Hu6b9Zc5cJkEswpoPU, KrazieKrush))"
2,1497568122350,2833,2525196351,CLASSIC,MATCHED_GAME,7.12.190.9002,11,"[(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1)), (2, (5_MnglWKGij6y...","[(11, BRONZE, [Row(masteryId=6111, rank=5), Row(masteryId=6121, rank=1), Row(masteryId=6134, rank=5), Row(masteryId=6142, rank=1), Row(masteryId=6312, rank=5), Row(masteryId=6323, rank=1), Row(masteryId=6331, rank=5), Row(masteryId=6343, rank=1),...",NA1,420,9,"[([Row(championId=17, pickTurn=1), Row(championId=51, pickTurn=2), Row(championId=105, pickTurn=3), Row(championId=86, pickTurn=4), Row(championId=72, pickTurn=5)], 0, 0, 3, False, True, True, False, False, True, 0, 0, 100, 4, 0, Fail), ([Row(cha...","(3, (gVUCaZcrTMk-NGBMvFxnh8aqYv-22QgoL1lpquT_J92a1ew, gVUCaZcrTMk-NGBMvFxnh8aqYv-22QgoL1lpquT_J92a1ew, NA1, /v1/stats/player_history/NA1/237529792, NA1, 1666, 747QTgKa2MFyw6NPPLA_lNuWnmXLYm1q3tZneodMAKJh-H8, Rhythm7))"
3,1497568122350,2833,2525196351,CLASSIC,MATCHED_GAME,7.12.190.9002,11,"[(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1)), (2, (5_MnglWKGij6y...","[(11, BRONZE, [Row(masteryId=6111, rank=5), Row(masteryId=6121, rank=1), Row(masteryId=6134, rank=5), Row(masteryId=6142, rank=1), Row(masteryId=6312, rank=5), Row(masteryId=6323, rank=1), Row(masteryId=6331, rank=5), Row(masteryId=6343, rank=1),...",NA1,420,9,"[([Row(championId=17, pickTurn=1), Row(championId=51, pickTurn=2), Row(championId

#### Add timestamp field

In [21]:
@udf(TimestampType())
def get_datetime_from(long_value):
    return datetime.fromtimestamp(long_value/1000)

In [22]:
raw_data = raw_data\
.withColumn('ts', get_datetime_from(raw_data.gameCreation))\
.withColumn('year', F.year('ts'))\
.withColumn('month', F.month('ts'))

In [23]:
raw_data.limit(1).toPandas()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams,ts,year,month
0,1497568122350,2833,2525196351,CLASSIC,MATCHED_GAME,7.12.190.9002,11,"[(1, (hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, hmjECtXPbAJB9QhyMeeCUJQu6i6gp0HHNQVfxEv_4qkOHJo, NA1, /v1/stats/player_history/NA1/200106415, NA1, 749, BDP8U_QMNal824JGX-cmavXAO75ad8wVywPWmKV5jqZaGwM, mohammadmatahen1)), (2, (5_MnglWKGij6y...","[(11, BRONZE, [Row(masteryId=6111, rank=5), Row(masteryId=6121, rank=1), Row(masteryId=6134, rank=5), Row(masteryId=6142, rank=1), Row(masteryId=6312, rank=5), Row(masteryId=6323, rank=1), Row(masteryId=6331, rank=5), Row(masteryId=6343, rank=1),...",NA1,420,9,"[([Row(championId=17, pickTurn=1), Row(championId=51, pickTurn=2), Row(championId=105, pickTurn=3), Row(championId=86, pickTurn=4), Row(championId=72, pickTurn=5)], 0, 0, 3, False, True, True, False, False, True, 0, 0, 100, 4, 0, Fail), ([Row(cha...",2017-06-15 20:08:42.350,2017,6


In [24]:
#write to parquet
raw_data = raw_data.repartition('year', 'month')
raw_data.write.partitionBy('year', 'month').mode('overwrite').parquet("data/parquet/matches_data")
raw_data_filtered = spark.read.parquet("data/parquet/matches_data")

In [25]:
raw_data_filtered.count()

4132

## Data Assessment

### Data schema

In [26]:
raw_data_filtered.printSchema()

root
 |-- gameCreation: long (nullable = true)
 |-- gameDuration: long (nullable = true)
 |-- gameId: long (nullable = true)
 |-- gameMode: string (nullable = true)
 |-- gameType: string (nullable = true)
 |-- gameVersion: string (nullable = true)
 |-- mapId: long (nullable = true)
 |-- participantIdentities: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- participantId: long (nullable = true)
 |    |    |-- player: struct (nullable = true)
 |    |    |    |-- accountId: string (nullable = true)
 |    |    |    |-- currentAccountId: string (nullable = true)
 |    |    |    |-- currentPlatformId: string (nullable = true)
 |    |    |    |-- matchHistoryUri: string (nullable = true)
 |    |    |    |-- platformId: string (nullable = true)
 |    |    |    |-- profileIcon: long (nullable = true)
 |    |    |    |-- summonerId: string (nullable = true)
 |    |    |    |-- summonerName: string (nullable = true)
 |-- participants: array (nullable = true)

#### Useful columns

- `gameCreation`:;
- `gameDuration`:;
- `gameId`:;
- `gameType`:;
- `gameVersion`:;
- `mapId`:;

### Data sample

In [27]:
raw_data_filtered.where(raw_data_filtered.gameId == 3474165150).toPandas()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams,ts,year,month


### Step 2: Explore and Assess the Data
#### Explore the Data 
> Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
> Document steps necessary to clean the data

In [28]:
raw_data_filtered = raw_data.withColumn(
    "team", F.explode(F.col("teams"))
).withColumn(
    "participant", F.explode(F.col("participants"))
).withColumn(
    "participantIdentity", F.explode(F.col("participantIdentities"))
).select(
    "gameCreation",
    "gameDuration",
    "gameId",
    "gameMode",
    "gameType",
    "team.teamId",
    "participant.participantId",
    "team.win",
    "team.firstBlood",
    "team.firstTower",
    "team.firstInhibitor",
    "team.firstBaron",
    "team.firstDragon",
    "team.firstRiftHerald",
    "team.towerKills",
    "team.baronKills",
    "team.dragonKills",
    "team.vilemawKills",
    "team.riftHeraldKills",
    "team.dominionVictoryScore",
    "participant.championId",
    "participant.spell1Id",
    "participant.spell2Id",
    F.col("participant.stats.win").alias("participant.stats.win"),
    "participant.stats.item0",
    "participant.stats.item1",
    "participant.stats.item2",
    "participant.stats.item3",
    "participant.stats.item4",
    "participant.stats.item5",
    "participant.stats.item6",
    "participant.stats.kills",
    "participant.stats.deaths",
    "participant.stats.assists",
    "participant.stats.largestKillingSpree",
    "participant.stats.largestMultiKill",
    "participant.stats.killingSprees",
    "participant.stats.longestTimeSpentLiving",
    "participant.stats.doubleKills",
    "participant.stats.tripleKills",
    "participant.stats.quadraKills",
    "participant.stats.pentaKills",
    "participant.stats.unrealKills",
    "participant.stats.totalDamageDealt",
    "participant.stats.magicDamageDealt",
    "participant.stats.physicalDamageDealt",
    "participant.stats.trueDamageDealt",
    "participant.stats.largestCriticalStrike",
    "participant.stats.totalDamageDealtToChampions",
    "participant.stats.magicDamageDealtToChampions",
    "participant.stats.physicalDamageDealtToChampions",
    "participant.stats.trueDamageDealtToChampions",
    "participant.stats.totalHeal",
    "participant.stats.totalUnitsHealed",
    "participant.stats.damageSelfMitigated",
    "participant.stats.damageDealtToObjectives",
    "participant.stats.damageDealtToTurrets",
    "participant.stats.visionScore",
    "participant.stats.timeCCingOthers",
    "participant.stats.totalDamageTaken",
    "participant.stats.magicalDamageTaken",
    "participant.stats.physicalDamageTaken",
    "participant.stats.trueDamageTaken",
    "participant.stats.goldEarned",
    "participant.stats.goldSpent",
    "participant.stats.turretKills",
    "participant.stats.inhibitorKills",
    "participant.stats.totalMinionsKilled",
    "participant.stats.neutralMinionsKilled",
    "participant.stats.neutralMinionsKilledTeamJungle",
    "participant.stats.neutralMinionsKilledEnemyJungle",
    "participant.stats.totalTimeCrowdControlDealt",
    "participant.stats.champLevel",
    "participant.stats.visionWardsBoughtInGame",
    "participant.stats.sightWardsBoughtInGame",
    "participant.stats.wardsPlaced",
    "participant.stats.wardsKilled",
    "participant.stats.firstBloodKill",
    "participant.stats.firstBloodAssist",
    "participant.stats.firstTowerKill",
    "participant.stats.firstTowerAssist",
    "participant.stats.firstInhibitorKill",
    "participant.stats.firstInhibitorAssist",
    F.col("participant.timeline.creepsPerMinDeltas.0-10").alias("creepsPerMinDeltas-0-10"),
    F.col("participant.timeline.creepsPerMinDeltas.10-20").alias("creepsPerMinDeltas-10-20"),
    F.col("participant.timeline.creepsPerMinDeltas.20-30").alias("creepsPerMinDeltas-20-30"),
    F.col("participant.timeline.creepsPerMinDeltas.30-end").alias("creepsPerMinDeltas-30-end"),
    F.col("participant.timeline.xpPerMinDeltas.0-10").alias("xpPerMinDeltas-0-10"),
    F.col("participant.timeline.xpPerMinDeltas.10-20").alias("xpPerMinDeltas-10-20"),
    F.col("participant.timeline.xpPerMinDeltas.20-30").alias("xpPerMinDeltas-20-30"),
    F.col("participant.timeline.xpPerMinDeltas.30-end").alias("xpPerMinDeltas-30-end"),
    F.col("participant.timeline.goldPerMinDeltas.0-10").alias("goldPerMinDeltas-0-10"),
    F.col("participant.timeline.goldPerMinDeltas.10-20").alias("goldPerMinDeltas-10-20"),
    F.col("participant.timeline.goldPerMinDeltas.20-30").alias("goldPerMinDeltas-20-30"),
    F.col("participant.timeline.goldPerMinDeltas.30-end").alias("goldPerMinDeltas-30-end"),
    F.col("participant.timeline.csDiffPerMinDeltas.0-10").alias("csDiffPerMinDeltas-0-10"),
    F.col("participant.timeline.csDiffPerMinDeltas.10-20").alias("csDiffPerMinDeltas-10-20"),
    F.col("participant.timeline.csDiffPerMinDeltas.20-30").alias("csDiffPerMinDeltas-20-30"),
    F.col("participant.timeline.csDiffPerMinDeltas.30-end").alias("csDiffPerMinDeltas-30-end"),
    F.col("participant.timeline.xpDiffPerMinDeltas.0-10").alias("xpDiffPerMinDeltas-0-10"),
    F.col("participant.timeline.xpDiffPerMinDeltas.10-20").alias("xpDiffPerMinDeltas-10-20"),
    F.col("participant.timeline.xpDiffPerMinDeltas.20-30").alias("xpDiffPerMinDeltas-20-30"),
    F.col("participant.timeline.xpDiffPerMinDeltas.30-end").alias("xpDiffPerMinDeltas-30-end"),
    F.col("participant.timeline.damageTakenPerMinDeltas.0-10").alias("damageTakenPerMinDeltas-0-10"),
    F.col("participant.timeline.damageTakenPerMinDeltas.10-20").alias("damageTakenPerMinDeltas-10-20"),
    F.col("participant.timeline.damageTakenPerMinDeltas.20-30").alias("damageTakenPerMinDeltas-20-30"),
    F.col("participant.timeline.damageTakenPerMinDeltas.30-end").alias("damageTakenPerMinDeltas-30-end"),
    F.col("participant.timeline.damageTakenDiffPerMinDeltas.0-10").alias("damageTakenDiffPerMinDeltas-0-10"),
    F.col("participant.timeline.damageTakenDiffPerMinDeltas.10-20").alias("damageTakenDiffPerMinDeltas-10-20"),
    F.col("participant.timeline.damageTakenDiffPerMinDeltas.20-30").alias("damageTakenDiffPerMinDeltas-20-30"),
    F.col("participant.timeline.damageTakenDiffPerMinDeltas.30-end").alias("damageTakenDiffPerMinDeltas-30-end"),
    "participant.timeline.role",
    "participant.timeline.lane",
    F.col("participantIdentity.participantId").alias("participantIdentity.participantId"),
    "participantIdentity.player.platformId",
    "participantIdentity.player.accountId",
    "participantIdentity.player.summonerId",
)

In [29]:
raw_data_filtered.count()

811040

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
> Map out the conceptual data model and explain why you chose that model

The goal of this data model is to provide a fact table with game stats and dimension tables with details about data referenced in the fact table.

![lol-conceptual-data-model](resources/img/lol-conceptual-data-model.png)

#### 3.2 Mapping Out Data Pipelines
> List the steps necessary to pipeline the data into the chosen data model
- First the data is persisted to S3 in parquet format for performance reasons;
- The raw data comes in a complex json structure in which would be very easy to work with once in Redshift. This goes in the opposite direction of having an easy-to-understand, and intuitive Data Warehouse.
- The transformations to achieve the final data model were performed with `pyspark.sql.functions.explode` function, which created different rows for items inside arrays, so data consumers would not have to navigate json structures. 

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
> Build the data pipelines to create the data model.

1. The data is extracted from riot's API (sourse);
1. Stages to S3 as-is;
1. A spark application consumes the data and performs transformations;
1. Loads it inti Lake Storage, S3, in parquet format;
1. Loads data from S3 into Redshift;

In [22]:
# Write code here

#### 4.2 Data Quality Checks
> Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
> * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
> * Unit tests for the scripts to ensure they are doing the right thing
> * Source/Count checks to ensure completeness
 
> Run Quality Checks

In [23]:
# Perform quality checks here

#### 4.3 Data dictionary 
> Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

There are two types of data dictionary:
- Active data dictionary: Created and maintained within the database. Updated automatically based on real data;
- Passive data dictionary: Created and maintained separate from the database. This type of document tends to get out of date as changes are not automatically replicated;

The data dictionary presented here is a passive one, as it resides out of the database.

##### Table: `dim_champion`


##### Table: `dim_item`


##### Table: `dim_summoner`


##### Table: `fact_game_match`

Field Name | Data Type | Field size for display | Description | Example
-----------|-----------|------------------------|-------------|--------
gameCreation | long | 10 | Timestamp of game creation | example
gameDuration | long | 10 | Duration of the match | example
gameId | long | 10 | Identifier for the game occurrence | example
gameMode | string | 10 | Classification of the match | example
gameType | string | 10 | Type of the game | example
teamId | long | 10 | Identifier the two teams in the match | example
participantId | long | 10 | Identifier for the player | [1-9]
win | string | 10 | Whether the team won the match | example
firstBlood | boolean | 10 | Whether the team dit the firstBlood | example
firstTower | boolean | 10 | Whether the team did the firstTower | example
firstInhibitor | boolean | 10 | Whether the team did the firstInhibitor | example
firstBaron | boolean | 10 | Whether the team did the firstBaron | example
firstDragon | boolean | 10 | Whether the team did the firstDragon | example
firstRiftHerald | boolean | 10 | firstRiftHerald | example
towerKills | long | 10 | towerKills | example
baronKills | long | 10 | baronKills | example
dragonKills | long | 10 | dragonKills | example
vilemawKills | long | 10 | vilemawKills | example
riftHeraldKills | long | 10 | riftHeraldKills | example
dominionVictoryScore | long | 10 | dominionVictoryScore | example
championId | long | 10 | championId | example
spell1Id | long | 10 | spell1Id | example
spell2Id | long | 10 | spell2Id | example
participant.stats.win | boolean | 10 | participant.stats.win | example
item0 | long | 10 | item0 | example
item1 | long | 10 | item1 | example
item2 | long | 10 | item2 | example
item3 | long | 10 | item3 | example
item4 | long | 10 | item4 | example
item5 | long | 10 | item5 | example
item6 | long | 10 | item6 | example
kills | long | 10 | kills | example
deaths | long | 10 | deaths | example
assists | long | 10 | assists | example
largestKillingSpree | long | 10 | largestKillingSpree | example
largestMultiKill | long | 10 | largestMultiKill | example
killingSprees | long | 10 | killingSprees | example
longestTimeSpentLiving | long | 10 | longestTimeSpentLiving | example
doubleKills | long | 10 | doubleKills | example
tripleKills | long | 10 | tripleKills | example
quadraKills | long | 10 | quadraKills | example
pentaKills | long | 10 | pentaKills | example
unrealKills | long | 10 | unrealKills | example
totalDamageDealt | long | 10 | totalDamageDealt | example
magicDamageDealt | long | 10 | magicDamageDealt | example
physicalDamageDealt | long | 10 | physicalDamageDealt | example
trueDamageDealt | long | 10 | trueDamageDealt | example
largestCriticalStrike | long | 10 | largestCriticalStrike | example
totalDamageDealtToChampions | long | 10 | totalDamageDealtToChampions | example
magicDamageDealtToChampions | long | 10 | magicDamageDealtToChampions | example
physicalDamageDealtToChampions | long | 10 | physicalDamageDealtToChampions | example
trueDamageDealtToChampions | long | 10 | trueDamageDealtToChampions | example
totalHeal | long | 10 | totalHeal | example
totalUnitsHealed | long | 10 | totalUnitsHealed | example
damageSelfMitigated | long | 10 | damageSelfMitigated | example
damageDealtToObjectives | long | 10 | damageDealtToObjectives | example
damageDealtToTurrets | long | 10 | damageDealtToTurrets | example
visionScore | long | 10 | visionScore | example
timeCCingOthers | long | 10 | timeCCingOthers | example
totalDamageTaken | long | 10 | totalDamageTaken | example
magicalDamageTaken | long | 10 | magicalDamageTaken | example
physicalDamageTaken | long | 10 | physicalDamageTaken | example
trueDamageTaken | long | 10 | trueDamageTaken | example
goldEarned | long | 10 | goldEarned | example
goldSpent | long | 10 | goldSpent | example
turretKills | long | 10 | turretKills | example
inhibitorKills | long | 10 | inhibitorKills | example
totalMinionsKilled | long | 10 | totalMinionsKilled | example
neutralMinionsKilled | long | 10 | neutralMinionsKilled | example
neutralMinionsKilledTeamJungle | long | 10 | neutralMinionsKilledTeamJungle | example
neutralMinionsKilledEnemyJungle | long | 10 | neutralMinionsKilledEnemyJungle | example
totalTimeCrowdControlDealt | long | 10 | totalTimeCrowdControlDealt | example
champLevel | long | 10 | champLevel | example
visionWardsBoughtInGame | long | 10 | visionWardsBoughtInGame | example
sightWardsBoughtInGame | long | 10 | sightWardsBoughtInGame | example
wardsPlaced | long | 10 | wardsPlaced | example
wardsKilled | long | 10 | wardsKilled | example
firstBloodKill | boolean | 10 | firstBloodKill | example
firstBloodAssist | boolean | 10 | firstBloodAssist | example
firstTowerKill | boolean | 10 | firstTowerKill | example
firstTowerAssist | boolean | 10 | firstTowerAssist | example
firstInhibitorKill | boolean | 10 | firstInhibitorKill | example
firstInhibitorAssist | boolean | 10 | firstInhibitorAssist | example
creepsPerMinDeltas-0-10 | double | 10 | creepsPerMinDeltas-0-10 | example
creepsPerMinDeltas-10-20 | double | 10 | creepsPerMinDeltas-10-20 | example
creepsPerMinDeltas-20-30 | double | 10 | creepsPerMinDeltas-20-30 | example
creepsPerMinDeltas-30-end | double | 10 | creepsPerMinDeltas-30-end | example
xpPerMinDeltas-0-10 | double | 10 | xpPerMinDeltas-0-10 | example
xpPerMinDeltas-10-20 | double | 10 | xpPerMinDeltas-10-20 | example
xpPerMinDeltas-20-30 | double | 10 | xpPerMinDeltas-20-30 | example
xpPerMinDeltas-30-end | double | 10 | xpPerMinDeltas-30-end | example
goldPerMinDeltas-0-10 | double | 10 | goldPerMinDeltas-0-10 | example
goldPerMinDeltas-10-20 | double | 10 | goldPerMinDeltas-10-20 | example
goldPerMinDeltas-20-30 | double | 10 | goldPerMinDeltas-20-30 | example
goldPerMinDeltas-30-end | double | 10 | goldPerMinDeltas-30-end | example
csDiffPerMinDeltas-0-10 | double | 10 | csDiffPerMinDeltas-0-10 | example
csDiffPerMinDeltas-10-20 | double | 10 | csDiffPerMinDeltas-10-20 | example
csDiffPerMinDeltas-20-30 | double | 10 | csDiffPerMinDeltas-20-30 | example
csDiffPerMinDeltas-30-end | double | 10 | csDiffPerMinDeltas-30-end | example
xpDiffPerMinDeltas-0-10 | double | 10 | xpDiffPerMinDeltas-0-10 | example
xpDiffPerMinDeltas-10-20 | double | 10 | xpDiffPerMinDeltas-10-20 | example
xpDiffPerMinDeltas-20-30 | double | 10 | xpDiffPerMinDeltas-20-30 | example
xpDiffPerMinDeltas-30-end | double | 10 | xpDiffPerMinDeltas-30-end | example
damageTakenPerMinDeltas-0-10 | double | 10 | damageTakenPerMinDeltas-0-10 | example
damageTakenPerMinDeltas-10-20 | double | 10 | damageTakenPerMinDeltas-10-20 | example
damageTakenPerMinDeltas-20-30 | double | 10 | damageTakenPerMinDeltas-20-30 | example
damageTakenPerMinDeltas-30-end | double | 10 | damageTakenPerMinDeltas-30-end | example
damageTakenDiffPerMinDeltas-0-10 | double | 10 | damageTakenDiffPerMinDeltas-0-10 | example
damageTakenDiffPerMinDeltas-10-20 | double | 10 | damageTakenDiffPerMinDeltas-10-20 | example
damageTakenDiffPerMinDeltas-20-30 | double | 10 | damageTakenDiffPerMinDeltas-20-30 | example
damageTakenDiffPerMinDeltas-30-end | double | 10 | damageTakenDiffPerMinDeltas-30-end | example
role | string | 10 | role | example
lane | string | 10 | lane | example
participantIdentity.participantId | long | 10 | participantIdentity.participantId | example
platformId | string | 10 | platformId | example
accountId | string | 10 | accountId | example
summonerId | string | 10 | summonerId | example

#### Step 5: Complete Project Write Up

##### Steps of this project
1. There is no games match's id made available by Riot's API, so this data had to be crawled. Riot requires an API key to allow game matches data to be fetched, so one had to be generated;
1. Wrote a python application to download champions, items, and game matches data from Riot's API, and save it to a S3 bucket;
1. Used a spark application to extract raw data from a S3 bucket, transformed it to a suitable Relational format, and loaded it back to S3 in parquet format;
1. Ran Data Definition Language statements to setup Redshift tables;
1. Transfered data from S3 to staging tables in Redshift with COPY statement;
1. Ran Data Manipulation Language statements to create dimension and fact tables from staging tables;
1. Ran Quality checks to ensure data has successfully been copied to right tables;
1. Ran Data Query Language statements to get analytical insignts;

##### Technology choices
> Clearly state the rationale for the choice of tools and technologies for the project.

- Processing Engine: Spark
    - Heavy processing was performed on the raw data in order to transform it to the final data model. Such a process would take too long if executed in a serial manner instead of parallelized with spark.
- Analytical database: Redshift
    - Each gaming match results in ~200 rows. If we consider that League of Legends has about [27 million players](https://www.unrankedsmurfs.com/blog/players-2017), and supposing that each player plays an average of 1 match per day, we would have. `27000000/10*200 = 540000000`
        - 27.000.000: 27 million players;
        - 10: 10 players per match;
        - 200: Number of records generated per match record;
        - 540.000.000: Total records in redshift (Five Hundred Forty Million)
        - Considering we have only in the `fact_game_match` table 120 columns, and most are INTEGER  (4 bytes), we would need at least 260GB of disk storage per DAY;

##### Data update frequency
> Propose how often the data should be updated and why

- As the goal of the datasource is to provide an analytical base for gaming strategies based on historical facts, new data can be appended to the dataset in a daily basis;
- Due to riot's API rate limit (100 req/2min) the ETL will be started once a day and run until there's no more data left to be fetched;

#### Approaching the problem in a larger scale
>- Write a description of how you would approach the problem differently under the following scenarios:
- The data was increased by 100x.
- The data populates a dashboard that must be updated on a daily basis by 7am every day.
- The database needed to be accessed by 100+ people.

#### References

- [Optimizing Performance](https://docs.aws.amazon.com/AmazonS3/latest/dev/optimizing-performance.html)
- [Hadoop Scalability and Performance Testing in Heterogeneous Clusters](https://www.researchgate.net/publication/291356207_Hadoop_Scalability_and_Performance_Testing_in_Heterogeneous_Clusters)
- [Scaling Uber’s Apache Hadoop Distributed File System for Growth](https://eng.uber.com/scaling-hdfs)
- [Data dictionary](https://www.tutorialspoint.com/What-is-Data-Dictionary)
- [Building An Analytics Data Pipeline In Python](https://www.dataquest.io/blog/data-pipelines-tutorial)
- [Redshift numeric types](https://docs.aws.amazon.com/redshift/latest/dg/r_Numeric_types201.html)
- [Pyspark extension types](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-types.html)

In [56]:
# COPY listing
# FROM 's3://mybucket/data/listings/parquet/'
# IAM_ROLE 'arn:aws:iam::0123456789012:role/MyRedshiftRole'
# FORMAT AS PARQUET;